# Visualization example

## Make a mosaic

In [2]:
!pip install cogeo-mosaic

     |████████████████████████████████| 2.0 MB 46.4 MB/s eta 0:00:01
  Created wheel for cogeo-mosaic: filename=cogeo_mosaic-3.0.0-py3-none-any.whl size=30357 sha256=9cf564efeafc013755f9a93881c8c844ceafdf8e4b73b28e50d41bbe94082064
  Stored in directory: /projects/.cache/pip/wheels/84/5e/a0/08e8fa5cc910de11b527d021c856feb8b73e18c7f655d688c0
Successfully built cogeo-mosaic
  Attempting uninstall: pygeos
    Found existing installation: pygeos 0.8
    Uninstalling pygeos-0.8:
      Successfully uninstalled pygeos-0.8
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [4]:
#items
orig = 'https://maap-ops-dataset.s3.amazonaws.com/bobafett/dps_output/latest/Landsat8_30542_comp_cog_2015-2020_dps.tif'
becomes = 's3://maap-ops-dataset/bobafett/dps_output/latest/Landsat8_30542_comp_cog_2015-2020_dps.tif'

#tiles = [
#    's3://maap-ops-dataset/bobafett/dps_output/latest/Landsat8_30542_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/bobafett/dps_output/latest/Landsat8_30543_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/bobafett/dps_output/latest/Landsat8_30822_comp_cog_2015-2020_dps.tif',
#    's3://maap-ops-dataset/bobafett/dps_output/latest/Landsat8_30823_comp_cog_2015-2020_dps.tif',
#]

tiles = [
    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30542_comp_cog_2015-2020_dps.tif',
    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30543_comp_cog_2015-2020_dps.tif',
    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30822_comp_cog_2015-2020_dps.tif',
    's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30823_comp_cog_2015-2020_dps.tif',
]


mosaicdata = MosaicJSON.from_urls(tiles, minzoom=9, maxzoom=16)


In [5]:
with MosaicBackend("/projects/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json", mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    
# s3://maap-ops-dataset/maap-users/alexdevseed/L8_tile_test.json

## Make a Map

In [10]:
import ipycmc

In [11]:
w = ipycmc.MapCMC()
w

MapCMC()

In [14]:
wmts_url = "https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = [
    "tile_format=png",
    "tile_scale=1",
    "pixel_selection=first",
    "TileMatrixSetId=WebMercatorQuad",
    "url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Flandsat8%2Fviz%2FL8_tile_test.json",
    "bidx=6",
    "resampling_method=nearest",
    "rescale=0%2C1",
    "return_mask=true",
    "colormap_name=viridis"
]
#TODO url encode so the params can be human readable
wmts_call = "?".join([wmts_url, "&".join([*params])])
wmts_call

'https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Flandsat8%2Fviz%2FL8_tile_test.json&bidx=6&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=viridis'

In [15]:
w.load_layer_config(wmts_call
                    , "wmts/xml")

In [16]:
import urllib

In [19]:
urllib.parse.unquote("s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Flandsat8%2Fviz%2FL8_tile_test.json")

's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json'